In [175]:
import numpy as np 
import pandas as pd
import scipy.linalg as slg
import matplotlib.pyplot as plt
import sklearn
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.datasets import make_classification
from sklearn import preprocessing
from sklearn import utils
from sklearn.impute import SimpleImputer
from scipy.linalg import cho_factor, cho_solve
import visualkeras
import time


---

## Import Data 

In [2]:
df = pd.read_csv('Life Expectancy Data.csv')
df.Status = df.Status.map({'Developing':0, 'Developed': 1})
df.columns = df.columns.str.replace(' ','')
df = df.interpolate(method = 'linear', limit_direction = 'forward')

In [3]:
df

,Country,Year,Status,Lifeexpectancy,AdultMortality,infantdeaths,Alcohol,percentageexpenditure,HepatitisB,Measles,...,Polio,Totalexpenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness1-19years,thinness5-9years,Incomecompositionofresources,Schooling
0,Afghanistan,2015,0,65.0,263.0,62,0.01,71.279624,65.0,1154,...,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,Afghanistan,2014,0,59.9,271.0,64,0.01,73.523582,62.0,492,...,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,Afghanistan,2013,0,59.9,268.0,66,0.01,73.219243,64.0,430,...,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,Afghanistan,2012,0,59.5,272.0,69,0.01,78.184215,67.0,2787,...,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,Afghanistan,2011,0,59.2,275.0,71,0.01,7.097109,68.0,3013,...,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933,Zimbabwe,2004,0,44.3,723.0,27,4.36,0.000000,68.0,31,...,67.0,7.13,65.0,33.6,454.366654,12777511.0,9.4,9.4,0.407,9.2
2934,Zimbabwe,2003,0,44.5,715.0,26,4.06,0.000000,7.0,998,...,7.0,6.52,68.0,36.7,453.351155,12633897.0,9.8,9.9,0.418,9.5
2935,Zimbabwe,2002,0,44.8,73.0,25,4.43,0.000000,73.0,304,...,73.0,6.53,71.0,39.8,57.348340,125525.0,1.2,1.3,0.427,10.0
2936,Zimbabwe,2001,0,45.3,686.0,25,1.72,0.000000,76.0,529,...,76.0,6.16,75.0,42.1,548.587312,12366165.0,1.6,1.7,0.427,9.8


---

## Data Cleaning

In [17]:
df_1 = df.iloc[:,1:22]    # Drop the Country Column
df_1

,Year,Status,Lifeexpectancy,AdultMortality,infantdeaths,Alcohol,percentageexpenditure,HepatitisB,Measles,BMI,...,Polio,Totalexpenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness1-19years,thinness5-9years,Incomecompositionofresources,Schooling
0,2015,0,65.0,263.0,62,0.01,71.279624,65.0,1154,19.1,...,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,2014,0,59.9,271.0,64,0.01,73.523582,62.0,492,18.6,...,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,2013,0,59.9,268.0,66,0.01,73.219243,64.0,430,18.1,...,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,2012,0,59.5,272.0,69,0.01,78.184215,67.0,2787,17.6,...,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,2011,0,59.2,275.0,71,0.01,7.097109,68.0,3013,17.2,...,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933,2004,0,44.3,723.0,27,4.36,0.000000,68.0,31,27.1,...,67.0,7.13,65.0,33.6,454.366654,12777511.0,9.4,9.4,0.407,9.2
2934,2003,0,44.5,715.0,26,4.06,0.000000,7.0,998,26.7,...,7.0,6.52,68.0,36.7,453.351155,12633897.0,9.8,9.9,0.418,9.5
2935,2002,0,44.8,73.0,25,4.43,0.000000,73.0,304,26.3,...,73.0,6.53,71.0,39.8,57.348340,125525.0,1.2,1.3,0.427,10.0
2936,2001,0,45.3,686.0,25,1.72,0.000000,76.0,529,25.9,...,76.0,6.16,75.0,42.1,548.587312,12366165.0,1.6,1.7,0.427,9.8


In [9]:
df_1['infantdeaths'].max()

1800

##### Sources: https://data.worldbank.org/indicator/SP.DYN.AMRT.FE?end=2020&most_recent_year_desc=false&start=2001&view=chart
##### Adult Mortality:The highest is 510/1000 and the lowest 1.2/1000
##### Infant Deaths: highest 1000 lowest 2 


---

##### Status:Developed or Developing status
##### Adult Mortality: Adult Mortality Rates of both sexes (probability of dying between 15 and 60 years per 1000 population)
##### infant deaths: Number of Infant Deaths per 1000 population
##### Alcohol: Alcohol recorded per capita (15+) consumption (in litres of pure alcohol)
##### Percentage expenditure: Expenditure on health as a percentage of Gross Domestic Product per capita(%)

---

#### Change some data to be more reseasonable 

In [12]:
df_1.AdultMortality = df_1.apply(lambda x: np.nan if (x.AdultMortality < 1.2 or x.AdultMortality > 510) else x.AdultMortality, axis=1)
df_1.infantdeaths = df_1.apply(lambda x: np.nan if (x.infantdeaths < 2 or x.infantdeaths > 600) else x.infantdeaths, axis=1)
df_1.BMI = df_1.apply(lambda x: np.nan if (x.BMI < 18 or x.BMI > 33) else x.BMI, axis=1)
df_1 = df_1.interpolate(method = 'linear', limit_direction = 'forward')

---

## Define X and Y

In [15]:
x1 = df_1.drop('Lifeexpectancy',axis=1)
y1 = df_1['Lifeexpectancy']
X1 = x1.to_numpy()
Y1 = y1.to_numpy() 
pd.DataFrame(X1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,2015.0,0.0,263.0,62.0,0.01,71.279624,65.0,1154.0,19.100000,83.0,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,2014.0,0.0,271.0,64.0,0.01,73.523582,62.0,492.0,18.600000,86.0,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,2013.0,0.0,268.0,66.0,0.01,73.219243,64.0,430.0,18.100000,89.0,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,2012.0,0.0,272.0,69.0,0.01,78.184215,67.0,2787.0,18.213043,93.0,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,2011.0,0.0,275.0,71.0,0.01,7.097109,68.0,3013.0,18.326087,97.0,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933,2004.0,0.0,40.0,27.0,4.36,0.000000,68.0,31.0,27.100000,42.0,67.0,7.13,65.0,33.6,454.366654,12777511.0,9.4,9.4,0.407,9.2
2934,2003.0,0.0,56.5,26.0,4.06,0.000000,7.0,998.0,26.700000,41.0,7.0,6.52,68.0,36.7,453.351155,12633897.0,9.8,9.9,0.418,9.5
2935,2002.0,0.0,73.0,25.0,4.43,0.000000,73.0,304.0,26.300000,40.0,73.0,6.53,71.0,39.8,57.348340,125525.0,1.2,1.3,0.427,10.0
2936,2001.0,0.0,73.0,25.0,1.72,0.000000,76.0,529.0,25.900000,39.0,76.0,6.16,75.0,42.1,548.587312,12366165.0,1.6,1.7,0.427,9.8


In [16]:
pd.DataFrame(Y1)

,0
0,65.0
1,59.9
2,59.9
3,59.5
4,59.2
...,...
2933,44.3
2934,44.5
2935,44.8
2936,45.3


---

## SVD for linear regression

In [18]:
train_df1 , test_df1 , y_train1 , y_test1 = train_test_split(X1,Y1,test_size=0.2,random_state=42)

In [181]:
t = time.time()    # time
u1 , s1 , vh1 = slg.svd(train_df1,full_matrices=False)    # SVD
svd_hat1 = vh1.T@np.diag(1/s1)@u1.T@y_train1
svd_pred1 = test_df1@svd_hat1
test_mse1 = np.mean((svd_pred1 - y_test1)**2)
print(np.round_(time.time()-t, 3),'sec elapsed') 
R_square1 = sklearn.metrics.r2_score(y_test1,svd_pred1)
print("MSE and R_Square:", "%.2f" % test_mse1,"%.2f" % R_square1)

0.004 sec elapsed
MSE and R_Square: 15.85 0.82


---

## Jacobi Method

In [21]:
def jacobi(A,x,b,n):             #  A is the input matrix, n is the iterative time, x is the guess
    D = np.diag(A)                  # find diag D
    M = A-D          # find L and U 
    for i in range(n):              # n time for interative
        x = slg.inv(D)@(b-np.dot(M,x))     # formulate in the class notes for interative
    return x

In [22]:
pd.DataFrame(train_df1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,2007.0,0.0,126.0,36.823529,5.28,345.463714,96.000000,0.0,25.500000,0.0,98.0,4.30,98.0,0.6,3112.285712,4.864380e+05,8.1,8.0,0.602,11.9
1,2001.0,0.0,152.0,16.705882,3.81,150.743486,32.333333,0.0,22.100000,0.0,91.0,5.19,9.0,0.8,1268.884564,4.437160e+05,9.4,9.3,0.562,11.0
2,2011.0,0.0,143.0,31.582090,10.43,0.000000,99.000000,0.0,21.093651,0.0,99.0,7.58,99.0,0.1,910.265223,1.021941e+05,4.3,4.3,0.733,12.9
3,2013.0,0.0,13.0,3.000000,1.29,594.645310,98.000000,16.0,18.445349,3.0,98.0,7.26,98.0,0.1,4199.472530,1.114558e+06,6.4,6.3,0.720,14.7
4,2002.0,0.0,95.0,80.030303,0.13,941.703687,99.000000,0.0,28.000000,0.0,99.0,3.40,94.0,0.1,16846.219800,3.684439e+07,6.7,6.1,0.820,13.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2345,2011.0,1.0,59.0,45.555556,6.91,3601.287457,82.000000,3.0,22.560000,0.0,96.0,9.60,96.0,0.1,22821.847000,4.162680e+05,0.8,0.7,0.826,14.8
2346,2010.0,0.0,287.0,4.000000,3.21,53.307827,83.000000,26.0,23.100000,7.0,82.0,6.70,83.0,5.9,543.957418,1.555880e+05,8.0,7.9,0.405,8.9
2347,2007.0,0.0,266.0,17.000000,6.08,56.778587,80.428571,0.0,18.565060,23.0,62.0,5.56,63.0,2.7,615.819819,9.556889e+06,4.2,4.2,0.458,8.4
2348,2003.0,1.0,72.0,2.000000,9.30,3519.258515,95.000000,10982.0,26.861386,3.0,97.0,8.17,96.0,0.1,27387.225800,5.731323e+06,0.5,0.5,0.841,15.4


In [23]:
def jacobi(A,x,b,n):             #  A is the input matrix, n is the iterative time, x is the guess
    D = np.diag(np.diag(A))                  # find diag D
    M = A-np.diagflat(D)          # find L and U 
    for i in range(n):              # n time for interative
        x = (b-np.dot(M,x))/D    # formulate in the class notes for interative
    return x

In [24]:
ans2 = jacobi(train_df1,np.ones(2350),y_train1,20)

ValueError: operands could not be broadcast together with shapes (2350,20) (400,400) 

In [26]:
D = np.diag(train_df1)              # find diag D

In [27]:
D.shape

(20,)

---

## Neural Networks

In [152]:
df_1

,Year,Status,Lifeexpectancy,AdultMortality,infantdeaths,Alcohol,percentageexpenditure,HepatitisB,Measles,BMI,...,Polio,Totalexpenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness1-19years,thinness5-9years,Incomecompositionofresources,Schooling
0,2015,0,65.0,263.0,62,0.01,71.279624,65.0,1154,19.1,...,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,2014,0,59.9,271.0,64,0.01,73.523582,62.0,492,18.6,...,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,2013,0,59.9,268.0,66,0.01,73.219243,64.0,430,18.1,...,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,2012,0,59.5,272.0,69,0.01,78.184215,67.0,2787,17.6,...,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,2011,0,59.2,275.0,71,0.01,7.097109,68.0,3013,17.2,...,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933,2004,0,44.3,723.0,27,4.36,0.000000,68.0,31,27.1,...,67.0,7.13,65.0,33.6,454.366654,12777511.0,9.4,9.4,0.407,9.2
2934,2003,0,44.5,715.0,26,4.06,0.000000,7.0,998,26.7,...,7.0,6.52,68.0,36.7,453.351155,12633897.0,9.8,9.9,0.418,9.5
2935,2002,0,44.8,73.0,25,4.43,0.000000,73.0,304,26.3,...,73.0,6.53,71.0,39.8,57.348340,125525.0,1.2,1.3,0.427,10.0
2936,2001,0,45.3,686.0,25,1.72,0.000000,76.0,529,25.9,...,76.0,6.16,75.0,42.1,548.587312,12366165.0,1.6,1.7,0.427,9.8


In [153]:
x2 = df_1.drop('Status',axis=1)
y2 = df_1['Status']
X2 = x1.to_numpy()
Y2 = y1.to_numpy()

In [154]:
def scaling(data):
    max = np.max(data,0)
    min = np.min(data,0)
    return (data-min)/(max-min)

Scaling data to range 0-1

In [155]:
X2_N = scaling(X2)
pd.DataFrame(X2_N)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1.000000,0.0,0.526210,0.104530,0.000000,0.003659,0.653061,0.005439,0.073333,0.0332,0.031250,0.452118,0.649485,0.000000,0.004889,0.026074,0.619565,0.603509,0.505274,0.487923
1,0.933333,0.0,0.542339,0.108014,0.000000,0.003774,0.622449,0.002319,0.040000,0.0344,0.572917,0.453279,0.618557,0.000000,0.005127,0.000253,0.630435,0.610526,0.502110,0.483092
2,0.866667,0.0,0.536290,0.111498,0.000000,0.003759,0.642857,0.002027,0.006667,0.0356,0.614583,0.450377,0.639175,0.000000,0.005287,0.024525,0.637681,0.617544,0.495781,0.478261
3,0.800000,0.0,0.544355,0.116725,0.000000,0.004014,0.673469,0.013135,0.014203,0.0372,0.666667,0.473012,0.670103,0.000000,0.005608,0.002857,0.644928,0.628070,0.488397,0.473430
4,0.733333,0.0,0.550403,0.120209,0.000000,0.000364,0.683673,0.014200,0.021739,0.0388,0.677083,0.435287,0.680412,0.000000,0.000519,0.002302,0.655797,0.635088,0.478903,0.458937
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933,0.266667,0.0,0.076613,0.043554,0.243561,0.000000,0.683673,0.000146,0.606667,0.0168,0.666667,0.392339,0.649485,0.663366,0.003799,0.009875,0.336957,0.326316,0.429325,0.444444
2934,0.200000,0.0,0.109879,0.041812,0.226764,0.000000,0.061224,0.004703,0.580000,0.0164,0.041667,0.356936,0.680412,0.724752,0.003790,0.009764,0.351449,0.343860,0.440928,0.458937
2935,0.133333,0.0,0.143145,0.040070,0.247480,0.000000,0.734694,0.001433,0.553333,0.0160,0.729167,0.357516,0.711340,0.786139,0.000467,0.000097,0.039855,0.042105,0.450422,0.483092
2936,0.066667,0.0,0.143145,0.040070,0.095745,0.000000,0.765306,0.002493,0.526667,0.0156,0.760417,0.336042,0.752577,0.831683,0.004589,0.009558,0.054348,0.056140,0.450422,0.473430


In [156]:
def sigmoid(data):
    return 1/(1+np.exp(-data)) 

In [157]:
def wb_calc(x,y,hidden_layers,alpha= 0.1,reg_lambda= 0):
    w1 = np.mat(np.random.randn(20,hidden_layers))
    b1 = np.mat(np.random.randn(1,hidden_layers))
    w2 = np.mat(np.random.randn(hidden_layers,1))
    b2 = np.mat(np.random.randn(1,1))
    #FP
    z1 = x*w1 + b1
    a1 = sigmoid(z1)
    z2 = a1*w2 + b2 
    a2 = sigmoid(z2)   # (2938, 1) 
    y1 = np.expand_dims(y,axis=1)
    #BP
    a0 = x
    delta2 = np.subtract(a2,y1)   # y=(2938,)
    delta1 = np.mat((delta2*w2.T).A*(a1.A*(1-a1).A))
    dw1 = a0.T*delta1 + reg_lambda*w1
    db1 = np.sum(delta1,0)
    dw2 = a1.T*delta2 + reg_lambda*w2
    db2 = np.sum(delta2,0)
    # re-entre w b
    w1 -= alpha *dw1
    b1 -= alpha *db1
    w2 -= alpha *dw2
    b1 -= alpha *db1
    return w1,b1,w2,b2

In [172]:
w1,b1,w2,b2 = wb_calc(X2_N,Y2,2,0.3)

In [173]:
print('1:',w1)
print('2:',b1)
print('3:',w2)
print('4:',b2)

1: [[ 2.88382535e+01 -1.57830654e+03]
 [ 1.55700306e+01 -1.11152558e+03]
 [ 2.70422956e+01 -8.70983780e+02]
 [ 5.16651572e+00 -2.18081737e+02]
 [ 2.19938615e+01 -1.00011724e+03]
 [ 1.30125349e+00 -2.06217348e+02]
 [ 5.56788935e+01 -2.97472450e+03]
 [ 4.92803094e+00 -2.40669591e+01]
 [ 3.42712378e+01 -1.13694329e+03]
 [ 4.74312400e+00 -2.71113718e+01]
 [ 5.82790709e+01 -2.94936431e+03]
 [ 2.58011161e+01 -1.18930844e+03]
 [ 6.09050283e+01 -2.96684453e+03]
 [ 2.20990585e+00 -4.84303103e+01]
 [ 4.24211398e+00 -2.95106430e+02]
 [ 5.16441299e-03 -2.35514168e+01]
 [ 1.96087386e+01 -4.13089221e+02]
 [ 2.04086858e+01 -3.99437155e+02]
 [ 4.60689911e+01 -2.46157112e+03]
 [ 4.27190723e+01 -2.14497280e+03]]
2: [[  168.39387756 -6921.96490353]]
3: [[  778.54244403]
 [54104.45818253]]
4: [[-0.05641762]]


---

## QR Decomposition

In [179]:
t = time.time()    # time
new_matrix = np.c_[X2,Y2]
Qq , Rr = slg.qr(new_matrix, mode='economic') 
R = Rr[:-1,:-1]         # class_note
r = Rr[:-1,-1]
beta_hat2 = slg.solve(R,r)   # beta_hat
print(np.round_(time.time()-t, 3),'sec elapsed')  
pd.DataFrame(beta_hat2)

0.003 sec elapsed


,0
0,2.596185e-02
1,1.342632e+00
2,-2.105841e-02
3,-3.187666e-03
4,6.514215e-02
5,2.470663e-04
6,-8.576419e-03
7,-3.593699e-05
8,1.589060e-01
9,-1.488602e-03


---

## Cholesky decomposition

In [182]:
t = time.time()                                         # time start
A1 = X2.T.dot(X2)
# cholesky decomposition
c, low = slg.cho_factor(A1)
b1 = np.identity(7)
A_inv = slg.cho_solve((c, low),b1)
# beta
beta_hat1 = A_inv.dot(X2.T).dot(Y2)       
print(np.round_(time.time()-t, 3),'sec elapsed')       # time end

ValueError: incompatible dimensions ((20, 20) and (7, 7))

---

## LU Decomposition

In [183]:
t = time.time()
beta_hat = np.linalg.solve(X2.T.dot(X2) ,X2.T.dot(Y2))   # beta hat given
print(np.round_(time.time()-t, 3),'sec elapsed')       # counts time
print(beta_hat)

0.002 sec elapsed
[ 2.59618498e-02  1.34263200e+00 -2.10584073e-02 -3.18766607e-03
  6.51421478e-02  2.47066294e-04 -8.57641929e-03 -3.59369861e-05
  1.58906000e-01 -1.48860150e-03  2.66993740e-02  6.64938117e-02
  4.33859518e-02 -6.61615308e-01  1.19530450e-05  5.09637514e-09
 -1.52294604e-01  5.00364710e-02  6.88855714e+00  6.55647482e-01]


---

## Linear model

---

## Other topics 